In [80]:
agg_df = pd.read_csv('triplets.csv')

,user_id,product_id,amount
0,100001,100131500000,1.000
1,100001,101508800000,1.000
2,100001,101510200000,1.000
3,100001,102006000000,1.000
4,100001,102703100000,1.000
...,...,...,...
794237,110000,260283104100,4.106
794238,110000,520699300000,1.000
794239,110000,700160600003,1.000
794240,110000,700160600005,1.000


In [38]:
import numpy as np

In [37]:
R_df = agg_df.pivot(index = 'user_id', columns ='product_id', values = 'amount').fillna(0)
R_df.head()

product_id,71201400505,74300100905,74300300505,74300300605,74300302105,74300302205,74300400805,74300500205,74300603005,74400100105,...,999999901197,999999901260,999999901273,999999901274,999999901284,999999901287,999999901288,999999901290,999999901291,999999901299
user_id,,,,,,,,,,,,,,,,,,,,,
100001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
100002,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
100003,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
100004,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
100005,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [54]:
R_df.values

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [40]:
R = R_df.values
user_ratings_mean = np.mean(R, axis = 1)
R_demeaned = R - user_ratings_mean.reshape(-1, 1)

In [46]:
from scipy.sparse.linalg import svds
U, sigma, Vt = svds(R_demeaned, k = 100)

In [47]:
sigma = np.diag(sigma)


In [ ]:
original_values = R_df.copy()

In [66]:
all_user_predicted_ratings = np.dot(np.dot(U, sigma), Vt) + user_ratings_mean.reshape(-1, 1)
all_user_predicted_ratings[all_user_predicted_ratings < 0] = 0
predicted_df = pd.DataFrame(all_user_predicted_ratings, index=R_df.index, columns=R_df.columns)


In [67]:
mask = (original_values == 0)
original_values[mask] = predicted_df[mask]
final_predictions = original_values

In [88]:
rows, cols = np.where(predicted_df > 1)
for row, col in zip(rows, cols):
    print(f"user_id: {predicted_df.index[row]}, product_id: {predicted_df.columns[col]}, value: {predicted_df.iloc[row, col]}")


user_id: 100001, product_id: 204000600400, value: 10.842622272050807
user_id: 100001, product_id: 264200301000, value: 1.9323301692271306
user_id: 100001, product_id: 271500513200, value: 1.0654618271420384
user_id: 100001, product_id: 274502502400, value: 1.8076404037962253
user_id: 100002, product_id: 114020100000, value: 1.673610023114202
user_id: 100002, product_id: 204016200000, value: 2.6853413654448164
user_id: 100002, product_id: 204115100000, value: 1.0559099849136417
user_id: 100002, product_id: 263330401000, value: 1.2097930256167133
user_id: 100002, product_id: 264500313200, value: 1.9695434973811534
user_id: 100003, product_id: 115002700000, value: 2.9782286769878863
user_id: 100003, product_id: 204115100000, value: 5.235342180440725
user_id: 100003, product_id: 205077200000, value: 1.958238564479394
user_id: 100003, product_id: 263500513200, value: 1.9511612947290444
user_id: 100004, product_id: 204115100000, value: 1.1476328222843888
user_id: 100004, product_id: 26428010

KeyboardInterrupt: 

In [98]:
final_predictions.loc[(100009, 270400509300)]

0.3145047797727468

In [99]:
agg_df[(agg_df.user_id == 100009) & (agg_df.product_id == 270400509300)]

,user_id,product_id,amount
